In [1]:
library(tidyverse)
library(caret)
library(randomForest)
library(doParallel)

options(digits=5)
options(warn=-1)
options(width=100)

R.Version()$version.string
print(map_chr(c('tidyverse','caret','randomForest'),
              function(x){paste(x,': ',packageVersion(x),', ',sep='')}), quote=F)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
√ ggplot2 3.1.1     √ purrr   0.3.2
√ tibble  2.1.1     √ dplyr   0.8.1
√ tidyr   0.8.3     √ stringr 1.4.0
√ readr   1.3.1     √ forcats 0.4.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Loading required package: lattice

Attaching package: 'caret'

The following object is masked from 'package:purrr':

    lift

randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: 'randomForest'

The following object is masked from 'package:dplyr':

    combine

The following object is masked from 'package:ggplot2':

    margin

Loading required package: foreach

Attaching package: 'foreach'

The following objects are masked from 'pa

[1] "R version 3.6.0 Patched (2019-04-28 r76433)"

[1] tidyverse: 1.2.1,      caret: 6.0.84,         randomForest: 4.6.14, 


In [2]:
#============================
# Read Data

train <- read_csv('pml-training.csv')
test  <- read_csv('pml-testing.csv' )

Parsed with column specification:
cols(
  .default = col_double(),
  user_name = col_character(),
  cvtd_timestamp = col_character(),
  new_window = col_character(),
  kurtosis_roll_belt = col_character(),
  kurtosis_picth_belt = col_character(),
  kurtosis_yaw_belt = col_character(),
  skewness_roll_belt = col_character(),
  skewness_roll_belt.1 = col_character(),
  skewness_yaw_belt = col_character(),
  max_yaw_belt = col_character(),
  min_yaw_belt = col_character(),
  amplitude_yaw_belt = col_character(),
  kurtosis_picth_arm = col_character(),
  kurtosis_yaw_arm = col_character(),
  skewness_pitch_arm = col_character(),
  skewness_yaw_arm = col_character(),
  kurtosis_yaw_dumbbell = col_character(),
  skewness_yaw_dumbbell = col_character(),
  kurtosis_roll_forearm = col_character(),
  kurtosis_picth_forearm = col_character()
  # ... with 8 more columns
)
See spec(...) for full column specifications.
Parsed with column specification:
cols(
  .default = col_logical(),
  X1 = col_do

In [3]:
#============================
# Pre Process Sample Date

train <- train %>% mutate(flag='train')
test  <- test  %>% mutate(flag='test' ) %>% 
          mutate(problem_id=as.character(problem_id)) %>% rename(classe=problem_id)

all  <- bind_rows(train,test) %>% select(names(train)[-c(1,3:7)])
drop <- map_lgl(all[,2:153], function(c){ any(is.na(c))} )      # Drop columns contain NA
all <- all[, c(T,!drop,T,T)]
print(ncol(all))

train <- all %>% filter(flag=='train') %>% select(-flag) %>% mutate(classe=factor(classe))
test  <- all %>% filter(flag=='test')  %>% select(-flag)

[1] 55


In [4]:
#============================
# Split Training Data to train, valid
set.seed(0)

index <- createDataPartition(train$classe, p=0.8, list=F, times=1)
train.t <- train[ index,]
train.v <- train[-index,]

In [5]:
#============================
# Fit randomForest
# 

set.seed(0)

    cl <- makeCluster(detectCores()) # for Parallel processing
    registerDoParallel(cl)           #
#-- Fit ----
modFit <- train(classe~., method='rf', data=train.t,
                trControl=trainControl(method = 'cv', number=10))
#-----------
    stopCluster(cl)                 #


In [8]:
# In sample
cat('=== In Sample ===')
confusionMatrix(predict(modFit,newdata=train.t), train.t$classe)
# Out of sample
cat('=== Out of Sample ===')
confusionMatrix(predict(modFit,newdata=train.v), train.v$classe)

=== In Sample ===

Confusion Matrix and Statistics

          Reference
Prediction    A    B    C    D    E
         A 4464    0    0    0    0
         B    0 3038    0    0    0
         C    0    0 2738    0    0
         D    0    0    0 2573    0
         E    0    0    0    0 2886

Overall Statistics
                                
               Accuracy : 1     
                 95% CI : (1, 1)
    No Information Rate : 0.284 
    P-Value [Acc > NIR] : <2e-16
                                
                  Kappa : 1     
                                
 Mcnemar's Test P-Value : NA    

Statistics by Class:

                     Class: A Class: B Class: C Class: D Class: E
Sensitivity             1.000    1.000    1.000    1.000    1.000
Specificity             1.000    1.000    1.000    1.000    1.000
Pos Pred Value          1.000    1.000    1.000    1.000    1.000
Neg Pred Value          1.000    1.000    1.000    1.000    1.000
Prevalence              0.284    0.194    0.174    0.164    0

=== Out of Sample ===

Confusion Matrix and Statistics

          Reference
Prediction    A    B    C    D    E
         A 1114    8    0    0    0
         B    2  749    1    0    0
         C    0    2  682    5    1
         D    0    0    1  637    1
         E    0    0    0    1  719

Overall Statistics
                                        
               Accuracy : 0.994         
                 95% CI : (0.992, 0.996)
    No Information Rate : 0.284         
    P-Value [Acc > NIR] : <2e-16        
                                        
                  Kappa : 0.993         
                                        
 Mcnemar's Test P-Value : NA            

Statistics by Class:

                     Class: A Class: B Class: C Class: D Class: E
Sensitivity             0.998    0.987    0.997    0.991    0.997
Specificity             0.997    0.999    0.998    0.999    1.000
Pos Pred Value          0.993    0.996    0.988    0.997    0.999
Neg Pred Value          0.999    0.997    0.999    0.99

In [7]:
#============================
# Predict from Test set

pred <- predict(modFit, newdata=test)
print(pred)

 [1] B A B A A E D B A A B C B A E E A B B B
Levels: A B C D E
